<a href="https://colab.research.google.com/github/aliciafmachado/sac/blob/final_version/notebooks/Run_SAC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook for running SAC

## Installations & imports

In [1]:
# Fixing the haiku problem
!pip install --upgrade pip
!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html

# Standard installs
!pip install dm-acme
!pip install dm-acme[reverb]
!pip install dm-acme[jax]
!pip install dm-acme[tf]
!pip install dm-acme[envs]
!pip install dm-env
!pip install dm-haiku
!pip install dm-tree
!pip install chex
!sudo apt-get install -y xvfb ffmpeg
!pip install imageio
!pip install gym
!pip install gym[classic_control]

# Need ml-collections for config file
!pip install ml_collections

!apt-get install x11-utils
!pip install pyglet

!pip install gym pyvirtualdisplay

from IPython.display import clear_output
clear_output()

In [2]:
# Workaround to make mujoco work on colab
# Source: https://gist.github.com/BuildingAtom/3119ac9c595324c8001a7454f23bf8c8

import os
if not os.path.exists('.mujoco_setup_complete'):
  # Get the prereqs
  ! apt-get -qq update
  ! apt-get -qq install -y libosmesa6-dev libgl1-mesa-glx libglfw3 libgl1-mesa-dev libglew-dev patchelf
  # Get Mujoco
  ! mkdir ~/.mujoco
  ! wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz -O mujoco.tar.gz
  ! tar -zxf mujoco.tar.gz -C "$HOME/.mujoco"
  ! rm mujoco.tar.gz
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  ! echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/.mujoco/mujoco210/bin' >> ~/.bashrc 
  ! echo 'export LD_PRELOAD=$LD_PRELOAD:/usr/lib/x86_64-linux-gnu/libGLEW.so' >> ~/.bashrc 
  # THE ANNOYING ONE, FORCE IT INTO LDCONFIG SO WE ACTUALLY GET ACCESS TO IT THIS SESSION
  ! echo "/root/.mujoco/mujoco210/bin" > /etc/ld.so.conf.d/mujoco_ld_lib_path.conf
  ! ldconfig
  # Install Mujoco-py
  ! pip install -U 'mujoco-py<2.2,>=2.1'
  # run once
  ! touch .mujoco_setup_complete

try:
  if _mujoco_run_once:
    pass
except NameError:
  _mujoco_run_once = False
if not _mujoco_run_once:
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  try:
    os.environ['LD_LIBRARY_PATH']=os.environ['LD_LIBRARY_PATH'] + ':/root/.mujoco/mujoco210/bin'
  except KeyError:
    os.environ['LD_LIBRARY_PATH']='/root/.mujoco/mujoco210/bin'
  try:
    os.environ['LD_PRELOAD']=os.environ['LD_PRELOAD'] + ':/usr/lib/x86_64-linux-gnu/libGLEW.so'
  except KeyError:
    os.environ['LD_PRELOAD']='/usr/lib/x86_64-linux-gnu/libGLEW.so'
  # presetup so we don't see output on first env initialization
  import mujoco_py
  _mujoco_run_once = True

Selecting previously unselected package libgl1-mesa-glx:amd64.
(Reading database ... 156271 files and directories currently installed.)
Preparing to unpack .../0-libgl1-mesa-glx_20.0.8-0ubuntu1~18.04.1_amd64.deb ...
Unpacking libgl1-mesa-glx:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Selecting previously unselected package libglew2.0:amd64.
Preparing to unpack .../1-libglew2.0_2.0.0-5_amd64.deb ...
Unpacking libglew2.0:amd64 (2.0.0-5) ...
Selecting previously unselected package libglew-dev:amd64.
Preparing to unpack .../2-libglew-dev_2.0.0-5_amd64.deb ...
Unpacking libglew-dev:amd64 (2.0.0-5) ...
Selecting previously unselected package libglfw3:amd64.
Preparing to unpack .../3-libglfw3_3.2.1-1_amd64.deb ...
Unpacking libglfw3:amd64 (3.2.1-1) ...
Selecting previously unselected package patchelf.
Preparing to unpack .../4-patchelf_0.9-1_amd64.deb ...
Unpacking patchelf (0.9-1) ...
Selecting previously unselected package libosmesa6:amd64.
Preparing to unpack .../5-libosmesa6_20.0.8-0ubuntu1~18.0

Clone the repo and install it:

In [3]:
! git clone https://github.com/aliciafmachado/sac.git

Cloning into 'sac'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (526/526), done.
remote: Compressing objects: 100% (362/362), done.
remote: Total 526 (delta 342), reused 310 (delta 152), pack-reused 0
Receiving objects: 100% (526/526), 410.53 KiB | 3.48 MiB/s, done.
Resolving deltas: 100% (342/342), done.


In [4]:
% cd sac

/content/sac


In [5]:
! pip install -e .

Obtaining file:///content/sac
  Preparing metadata (setup.py) ... done
  Running setup.py develop for sac-jax


## Training the model

In [12]:
! python src/main.py --config src/configs/reacher.py --seed 1 --experiment test

2022-04-01 19:07:13.373256: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running SAC on ReacherEnv
Model will be saved in results/test
Saving the model will overwrite folder named results/test. Continue (y/n)?y
I0401 19:07:16.282089 140088987953024 xla_bridge.py:253] Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
2022-04-01 19:07:16.293819: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I0401 19:07:16.294106 140088987953024 xla_bridge.py:253] Unable to initialize backend 'gpu': FAILED_PRECONDITION: No visible GPU devices.
I0401 19:07:16.294782 140088987953024 xla_bridge.py:253] Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
W0401 19:07:16.294952 140088987953024 xla_bridge.py:257] No GPU/TPU found, 

## Evaluating the model

In [13]:
! python src/eval.py --agent sac --mm_pth ! python src/eval.py --agent sac --mm_pth results/test/test_mm.pickle

2022-04-01 19:08:17.620189: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I0401 19:08:19.405655 139675943122816 xla_bridge.py:253] Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
2022-04-01 19:08:19.418047: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I0401 19:08:19.418330 139675943122816 xla_bridge.py:253] Unable to initialize backend 'gpu': FAILED_PRECONDITION: No visible GPU devices.
I0401 19:08:19.418978 139675943122816 xla_bridge.py:253] Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
W0401 19:08:19.419136 139675943122816 xla_bridge.py:257] No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
Evaluating sac agent on env ReacherEnv
Reward on simulation 1: 